In [1]:
from flask_cors import CORS,cross_origin
from flask import Flask, request, jsonify,Response ,make_response, send_file, send_from_directory
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from PIL import Image
import io
from io import BytesIO
import zipfile
import os
# from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
import face_recognition
import keras
from keras.models import load_model
import cv2


UPLOAD_FOLDER = 'userimages/'
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
counter = 0

class RequestUtils:
    def __init__(self, req : object) -> None:
        self.req = req
    
    def getFilenameByName(self, name : str) -> str:
        return self.req.files[name].filename
    
    def readFileByName(self, name : str) -> str:
        return self.req.files[name]

    def getReadObjByName(self, name : str) -> str:
        return self.req.files[name]
    
    def getRequestRealIP(self):
        return self.req.headers['X-Real-IP']
    
    def getQueryParam(self, name):
        return self.req.args.get(name)

# ipaddress:port/endpoint/
@app.route("/endpoint/", methods=['GET', 'POST'])
@cross_origin()
def index():
    response = make_response()
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add('Access-Control-Allow-Headers', "*")
    response.headers.add('Access-Control-Allow-Methods', "*")
    # print(request.files['zip_file'])
    req = RequestUtils(request)

    if request.method == 'POST':
        try:
            # file = request.files['img_file']
            file = req.getReadObjByName('img_file')
            file.save('temp_imgfile.jpg')

            images=[]
            # image_path = "C:/Users/Arihant/Desktop/duplicate/data"

            # os.mkdir(image_path)
            
            # with zipfile.ZipFile('temp_zipfile.zip', mode='r') as zip_ref:
            #     zip_ref.extractall('temp')
            path_og = "/Users/arihant/Desktop/Py/pishu/emo/inputpics"
            directory='/Users/arihant/Desktop/Py/pishu/emo/savedImages'
            # directory='users/savedImages'
                # for file in zip_ref.infolist():
                #     print(file.filename)
                #     image = Image.open(file)
                #     image = image.save(f"{image_path}/{file.filename}")

                    # image=Image.open(file)
                    # images.append(image)
            model= load_model("/Users/arihant/Desktop/Py/pishu/emo/model_v6_23.hdf5")
            def preprocess(face_image):
                face_image = cv2.resize(face_image, (48,48))
                face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])
                return face_image

            emotion_dict= {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}
            # inputimage = face_recognition.load_image_file("/Users/arihant/Desktop/Py/pishu/duplicate/content/101D3200/DSC_1681.JPG")
            inputimage = face_recognition.load_image_file('temp_imgfile.jpg')
            face_locations=face_recognition.face_locations(inputimage)
            c=1
            filename=''
            
            res_dict={}
            path_og='/Users/arihant/Desktop/Py/pishu/emo'
            directory='/Users/arihant/Desktop/Py/pishu/emo/savedImages'
            for face in face_locations:
                top, right, bottom, left = face
                face_image = inputimage[top:bottom, left:right]
                # plt.imshow(face_image)
                face_image_img=Image.fromarray(face_image)
                face_image_img.save("image_{0}.jpg".format(c))
                face_image_cv=cv2.imread("image_{0}.jpg".format(c))
                # plt.imshow(face_image_cv[:, :, : : -1])
                img_proc= preprocess(face_image_cv)
                predicted_class = np.argmax(model.predict(img_proc))
                label_map = dict((v,k) for k,v in emotion_dict.items()) 
                predicted_label = label_map[predicted_class]
                res_dict["image_{0}.jpg".format(c)]=predicted_label
                # print(predicted_label)
                os.chdir(directory)
                # face_image_img.save("{0}_image{1}.jpg".format(predicted_label,c))

                os.chdir(path_og)
                c+=1    

            res_list=[]
            for key,values in res_dict.items():
                    print(key," : ", values, "\n")
                    stringans=""
                    stringans+=key
                    stringans+=' : '
                    stringans+= values
                    res_list.append(stringans)
            res_tup=tuple(res_list)


            export_dir = "/exp"

            # Export the dataset
            # dataset.export(export_dir=export_dir, dataset_type=fo.types.ImageDirectory)
            zipObj = zipfile.ZipFile('output.zip', 'w')
            # zipObj.write(export_dir)

            # for x in os.listdir(export_dir):
            #     if (x.endswith(".JPG")):
            #         zipObj.write(x)

            pics= Path(directory).glob('*.jpg')
            for x in pics:
                zipObj.write(x)
            #     print(type(x))

            # close the Zip File
            # zipObj.close()
            send_file(zipObj,
                      mimetype = 'zip',
                      download_name='output.zip', 
                      as_attachment = True)

            zipObj.close()
            # for key,values in res_dict.items():
            resp={"Emotions": res_tup}

        except Exception as e:
            resp = {'Message': f'Exception, {e}'}
            print(f'exception: {e}')

        # del req
        return jsonify(resp)
        

        # return send_file(export_dir,
        #                  mimetype = 'zip',
        #                  download_name='output.zip',
        #                  as_attachment = True)
    
    return """
        <html><body>
            <h2>Upload file</h2>
            <form action="" method="post" enctype="multipart/form-data">
            enter image file : <input type="file" name="img_file" /><br />
            <input type="submit" />
            </form>
        </body></html>
        """

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8080, threaded=True)
    cors = CORS(app,resources={"/endpoint/": {"origins": "*"}})
    app.config['CORS_HEADERS'] = 'Content-Type'

# C:/Users/Arihant/Desktop/duplicate/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.29.42:8080
Press CTRL+C to quit
127.0.0.1 - - [19/Jan/2023 15:10:44] "GET /endpoiny/ HTTP/1.1" 404 -
127.0.0.1 - - [19/Jan/2023 15:10:53] "GET /endpoint/ HTTP/1.1" 200 -
2023-01-19 15:11:15.390365: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 12ms/step


127.0.0.1 - - [19/Jan/2023 15:11:15] "POST /endpoint/ HTTP/1.1" 200 -


image_1.jpg  :  Surprise 

image_2.jpg  :  Happy 

image_3.jpg  :  Surprise 

image_4.jpg  :  Surprise 

image_5.jpg  :  Surprise 

